In [ ]:
import os
os.chdir('/content/drive/MyDrive/fb_college/traid')
os.system('pip uninstall networkx --y')
os.system('pip uninstall python-louvain --y')
os.system('pip uninstall python-louvain --y')
os.system('pip install -r requirements.txt')

0

In [ ]:
import igraph as ig
import os
import pandas as pd
import numpy as np
import datetime as dt
import time
import community.community_louvain as community_louvain
from cdlib import algorithms
import networkx as nx
import collections
import sys
import psutil
import concurrent.futures
from multiprocessing import Pool
from multiprocessing import cpu_count
import multiprocessing
import swifter
import argparse
import json
from pathlib import Path

In [ ]:
def parse_args():
  parser = argparse.ArgumentParser(
      description='compute online var base on all tie and firstday'
  )
  parser.add_argument(
      '--ch_dir',
      type=str,
      default='/content/drive/MyDrive/fb_college/traid'
  )
  parser.add_argument(
      '--dtype_file',
      type=str,
      default='./tmp/dtype.json'
  )
  parser.add_argument(
      '--parse_dates_file',
      type=str,
      default='./tmp/parse_dates.json'
  )
  parser.add_argument(
      '--time_range',
      type=list,
      help='research time range',
      default=['2012-09-30', '2015-06-30'],
  )
  parser.add_argument(
      '--res_filt',
      type=float,
      help='filt response rate',
      default=0.7,
  )
  parser.add_argument(
      '--tie_mode',
      type=str,
      help='tie type, if departclass then only analysis in class tie',
      default='departclass',
  )
  parser.add_argument(
      '--var_level',
      type=str,
      help='compute tie level, can chose ego, d1 or d2',
      default='d1',
  )
  # --------------------------> raw data file
  parser.add_argument(
      '--survey_file',
      type=str,
      help='raw survey file',
      default='../raw/csv/survey.csv'
  )
  # --------------------------> preprocess file
  parser.add_argument(
      '--all_tie_file',
      type=str,
      default='./tmp/all_tie_departclass.csv',
  )
  parser.add_argument(
      '--egolist_file',
      type=str,
      help='research ego list, default is from the class reponse rate > 0.7',
      default='./tmp/egores0.7.csv'
  )
  parser.add_argument(
      '--egofirstime_file',
      type=str,
      help='firstime ego use face book',
      default='./tmp/ego_firstime.csv'
  )
  parser.add_argument(
      '--d1firstday_file',
      type=str,
      help='d1 first time',
      default='./tmp/d1firstday_departclass.csv'
  )
  parser.add_argument(
      '--d2firstday_file',
      type=str,
      help='d1 first time',
      default='./tmp/d2firstday_departclass.csv'
  )
  parser.add_argument(
      '--d2d1ego_file',
      type=str,
      help='d2-d1 level contain their ego',
      default='./tmp/d2d1ego_departclass.csv'
  )
  parser.add_argument(
      '--output_dir',
      type=str,
      help='d2-d1 level contain their ego',
      default='./tmp/',
  )
  parser.add_argument(
      '--isDebug',
      type=bool,
      help='if true, only read 100 line',
      default=False,
  )
  parser.add_argument(
      '--debug_n',
      type=int,
      help='if in debug mode read n rows',
      default=20,
  )
  args, unknown = parser.parse_known_args()
  return args
args = parse_args()

In [ ]:
# set up working dir
os.chdir(args.ch_dir)
# from utils import onlineVar # 自定義自定義package
# set up dataframe display format
# read dtype and time columns file
dtype = json.loads(Path(args.dtype_file).read_text())
parse_dates = json.loads(Path(args.parse_dates_file).read_text())

# read file

In [ ]:
# tie file
usecols = ['from_id_p', 'from_id', 'createdtime_date',
           'tie_type', 'tie_file', 'tag_or_com', 'from_id_departclass', 'from_id_p_departclass']
parse_dates_col = [parse_dates[args.all_tie_file.split('/')[-1]]] # 取出檔名，查詢這個檔名的時間欄位名稱
tie_df = pd.read_csv(args.all_tie_file,
                     dtype=dtype,
                     parse_dates=parse_dates_col,
                     usecols=usecols)
# read alter_df file
if (args.var_level == 'ego') or (args.var_level == 'd2'):
  alter_file = args.d2firstday_file
  usecols = ['ego_id' ,'d2_alterid', 'd2firstday',
           'eventover', 'transferday', 'egofirstime']
elif (args.var_level == 'd1'):
  alter_file = args.d2d1ego_file
  usecols = ['ego_id' ,'d2_alterid', 'd1_alterid', 'd2firstday', 'd1firstday',
             'createdtime_date', 'egofirstime', 'eventover']

parse_dates_col = parse_dates[alter_file.split('/')[-1]]

if args.isDebug:
  nrows=args.debug_n
else:
  nrows=None

alter_df = pd.read_csv(alter_file,
                    parse_dates=parse_dates_col,
                    dtype=dtype,
                    usecols=usecols,
                    nrows=nrows)
print(tie_df.shape)

(683065, 8)


In [ ]:
# set dt time index and sort by id str value and time(from earlist to latest)
def add_index(dt, cluster_id_col, time_col):
  dt = dt.reset_index(drop=True)
  dt = dt.sort_values(by=[time_col])
  # dt = dt.sort_values(by=[cluster_id_col, time_col])
  # dt.set_index([cluster_id_col,
  #               pd.DatetimeIndex(dt[time_col])],
  #              inplace=True)
  dt.set_index([pd.DatetimeIndex(dt[time_col])],
                inplace=True)
  return dt

# 2min
# partition node and return two dataframe
# node_id, the partition it belong
# partition id, how many node it have
def partition(nx):
  partition_dic = community_louvain.best_partition(nx)
  # partition_df = pd.DataFrame.from_dict(partition_dic, orient='index')
  # count the number of node in each partition
  values = [partition_dic.get(node) for node in nx.nodes()]
  counter_dic = collections.Counter(values)
  # counter_df = pd.DataFrame.from_dict(counter_dic, orient='index')
  return partition_dic, counter_dic

def nx_in_time(dt, start, end, time_col, node_cols):
  # sort the date from  earliest to the latest
  dt = dt.sort_values(by=[time_col])
  dt = dt.set_index([pd.DatetimeIndex(dt[time_col])])
  # cut time range
  dt = dt[start:end]
  # drop the duplicated tie
  dt = dt[node_cols]
  dt = dt.reset_index(drop=True)
  dt.drop_duplicates()
  # convert from dt to nx object
  g = nx.from_pandas_edgelist(dt, source=node_cols[0], target=node_cols[1])
  return g

 # 9s
# created a no-direction network
# sort node by str value and drop the duplicates tie
def no_direct(dt):
  dt['big_id'] = np.where(dt['from_id'] > dt['from_id_p'],
                          dt['from_id'], dt['from_id_p'])
  dt['small_id'] = np.where(dt['from_id'] > dt['from_id_p'],
                            dt['from_id_p'], dt['from_id'])
  # only keep the columns we need
  dt = dt[['big_id', 'small_id', 'createdtime_date']]
  dt = dt.drop_duplicates()
  return dt

# 49s
def add_cluster_col(dt, node_cols):
  add_cluster_cols = []
  for node in node_cols:
    add_cluster_col = str(node) + '_group'
    dt[add_cluster_col] = dt[node].apply(lambda id: cluster_id_dic[id])
    add_cluster_cols.append(add_cluster_col)
  if len(node_cols) == 2:
    # if two node belongs to diffetent cluster, record two cluster
    dt['tie_cluster_set'] = np.where(
        dt[add_cluster_cols[0]].astype('str') != dt[add_cluster_cols[1]].astype('str'),
        dt[add_cluster_cols[0]].astype('str') + ',' + dt[add_cluster_cols[1]].astype('str'),
        dt[add_cluster_cols[0]].astype('str')) # else just record one cluster
    dt['tie_cluter_num'] = np.where( # if two node belongs to diffetent cluster
        dt[add_cluster_cols[0]].astype('str') != dt[add_cluster_cols[1]].astype('str'),
        2, 1) # there have tow cluster, else there is only 1
  return dt

def add_cluster_col_to_group_df(dt, value_vars):
  # the tie only belongs to one cluster
  cluster1_dt = dt[dt['tie_cluter_num'] == 1]
  cluster1_dt = cluster1_dt.drop(labels=value_vars[0], axis=1
                              ).rename(columns={value_vars[1]:'group_id'})
  # the tie only belongs to two cluster
  cluster2_df = dt[dt['tie_cluter_num'] == 2]
  # generate a new columns 0 1 2 3 4 5...
  cluster2_df['id'] = range(len(cluster2_df))
  reshape_dt = pd.melt(cluster2_df, id_vars='id', value_vars=value_vars)
  reshape_dt = reshape_dt.merge(cluster2_df, on='id', indicator=True)
  # rename and drop some columns to concat
  reshape_dt = reshape_dt = reshape_dt.drop(
      labels=['variable', 'big_id_group', 'small_id_group', '_merge', 'id'],
      axis=1).rename(columns={'value':'group_id'})
  add_cluster_dt = pd.concat([reshape_dt, cluster1_dt])
  return add_cluster_dt

# use index reference + swifter ok version 
# in colab with swifater 1000 7min
# # in colab ddf partition=5 1000 182s
# def lcc(alter_id, alter_group_col, start, end,
#         tie_dt, tie_group_col, tie_id_cols):
#   # if one of the person in tie_dt, his/her group number = alter group number
#   # combine it into nw_dt
#   # nw_dt = tie_dt.loc[alter_group_col].loc[start:end]
#   # print(tie_dt[start:end])
#   # nw_dt = tie_dt[start:end]
#   # nw_dt = tie_dt
#   # set the time range, only pick the tie from sart time to end time
#   # created a network object
#   nw = nx.from_pandas_edgelist(nw_dt, source=tie_id_cols[0],
#                                target=tie_id_cols[1])
#   lcc = nx.clustering(nw, alter_id)
#   return lcc

In [ ]:
# use index reference + swifter ok version 
# in colab with swifater 1000 7min
# in colab ddf partition=5 1000 182s
def lcc(alter_id, alter_group_col, start, end,
        tie_dt, tie_group_col, tie_id_cols):
  # if one of the person in tie_dt, his/her group number = alter group number
  # combine it into nw_dt
  #nw_dt = tie_dt.loc[alter_group_col].loc[start:end] 
  nw_dt = tie_dt.loc[start:end]
  # set the time range, only pick the tie from sart time to end time
  # created a network object
  nw = nx.from_pandas_edgelist(nw_dt, source=tie_id_cols[0],
                               target=tie_id_cols[1])
  lcc = nx.clustering(nw, alter_id)
  return lcc
def add_lcc(dt, alter_id_col, alter_group_col, start_col, end_col,
            tie_dt, tie_group_col, tie_id_cols, lcc_col):
  dt[lcc_col] = dt.swifter.apply(
    lambda row: lcc(row[alter_id_col], row[alter_group_col],
                    row[start_col], row[end_col],
                    tie_dt, tie_group_col, tie_id_cols), axis=1)
  return dt

In [ ]:
FB_EARLEST = '1990-01-01'
FB_LATEST = '2017-12-30'

if (args.var_level == 'd1'):
  alter_id = 'd1_alterid'
  var_duration = ['d1firstday', 'eventover']
elif (args.var_level == 'd2'):
  alter_id = 'd2_alterid'
  var_duration = ['d2firstday', 'eventover']
elif (args.var_level == 'ego'):
  alter_id = 'ego_id'
  var_duration = ['egofirstime', 'd2firstday']

time_col = 'createdtime_date'
new_lcc_col = f'{args.var_level}_lcc'

In [ ]:
# gen no direction tie form
tie_df = no_direct(tie_df)

all_nx = nx_in_time(tie_df,
                    FB_EARLEST, FB_LATEST, time_col,
                    ['big_id', 'small_id'])
cluster_id_dic, cluter_ct_dic = partition(all_nx)

tie_df = add_cluster_col(tie_df, ['big_id', 'small_id'])
alter_df = add_cluster_col(alter_df, [alter_id])

group_tie_df = add_cluster_col_to_group_df(tie_df, ['big_id_group', 'small_id_group'])
group_tie_df = add_index(group_tie_df, 'group_id', time_col)


In [ ]:
alter_df = add_lcc(alter_df, alter_id, f'{alter_id}_group', var_duration[0], var_duration[1],
                group_tie_df, 'group_id', ['big_id', 'small_id'], f'{args.var_level}_lcc')
alter_df.to_csv(f'{args.output_dir}{args.var_level}_lcc.csv', index=False)

Pandas Apply:   0%|          | 0/124925 [00:00<?, ?it/s]